# Домашнее задание: Изображения как данные + препроцессинг (OpenCV)
Цель: собрать **минимальный, корректный и проверяемый** препроцессинг-пайплайн.

Что вы сделаете:
1) прочитаете изображение и корректно конвертируете BGR→RGB
2) реализуете resize с сохранением пропорций + pad (letterbox)
3) переведёте в float32 CHW и нормализуете (ImageNet mean/std)
4) соберёте eval-пайплайн и sanity-report

Баллы считаются локально в ноутбуке (TOTAL POINTS / 100).

In [ ]:
# @title 1) Student Info & Config
STUDENT_NAME = ""  # @param {type:"string"}
GROUP = ""         # @param {type:"string"}
ASSIGMENT = "CV_OPENCV_PREPROCESSING"
SEED = 42
START_DATE = "2026-02-23"
DUE_DATE = "2026-03-02"

In [ ]:
assert STUDENT_NAME.strip() != "", "Заполните STUDENT_NAME"
assert GROUP.strip() != "", "Заполните GROUP"
print('✔ Student Info OK')
print('Assignment:', ASSIGMENT)


In [ ]:
from datetime import datetime, timedelta, timezone
try:
    from zoneinfo import ZoneInfo
    MSK = ZoneInfo('Europe/Moscow')
except Exception:
    MSK = timezone(timedelta(hours=3))

def _sec(td):
    return float(td.total_seconds())

# Окно сдачи (по датам из первой ячейки)
start_dt = datetime.fromisoformat(START_DATE).replace(tzinfo=MSK)
due_dt   = datetime.fromisoformat(DUE_DATE).replace(tzinfo=MSK) + timedelta(hours=23, minutes=59, seconds=59)
# Время сдачи — текущее время (можно заменить на mtime файла, если хотите)
submission_dt = datetime.now(MSK)
print('START:', start_dt)
print('DUE  :', due_dt)
print('SUB  :', submission_dt)


In [ ]:
# Global score storage (do not modify)
SCORES = {}
total = 0

def _set_score(task_name, points, max_points):
    global total
    SCORES[task_name] = float(min(points, max_points))
    total = float(sum(SCORES.values()))
    print(f"Task {task_name}: {SCORES[task_name]} / {max_points} points")
    print(f"TOTAL POINTS: {total} / 100\n")


## 0) Импорты и генерация тестовых изображений
Чтобы не скачивать данные, мы создаём несколько картинок прямо в ноутбуке.

In [ ]:
import os
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt

DATA_DIR = Path('data')
DATA_DIR.mkdir(exist_ok=True)

def show(img_rgb, title=None):
    plt.figure(figsize=(4,4))
    if title:
        plt.title(title)
    plt.axis('off')
    plt.imshow(img_rgb)
    plt.show()

# 1) Цветовые блоки (проверка каналов)
h, w = 120, 180
img = np.zeros((h, w, 3), dtype=np.uint8)
img[:, :60]    = (0, 0, 255)   # red in RGB
img[:, 60:120] = (0, 255, 0)   # green
img[:, 120:]   = (255, 0, 0)   # blue
cv2.imwrite(str(DATA_DIR / 'color_blocks.png'), img)

# 2) Прямоугольник с аспектом (для letterbox)
h2, w2 = 90, 260
rect = np.full((h2, w2, 3), 255, dtype=np.uint8)
cv2.rectangle(rect, (20, 15), (240, 75), (0, 0, 0), 3)
cv2.putText(rect, 'ASPECT', (40, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,0), 3, cv2.LINE_AA)
cv2.imwrite(str(DATA_DIR / 'aspect.png'), rect)

print('Generated:', [p.name for p in DATA_DIR.iterdir()])

In [ ]:
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)


## Task 1 (20 баллов): прочитать изображение и конвертировать BGR→RGB
Реализуйте функцию `read_rgb(path)`:
- читает картинку через `cv2.imread`
- конвертирует в RGB (`cv2.COLOR_BGR2RGB`)
- возвращает `np.ndarray` типа `uint8` с shape `(H, W, 3)`

In [ ]:
# TODO: implement read_rgb
def read_rgb(path: str) -> np.ndarray:
    """Read image with OpenCV and return RGB uint8 HWC."""
    # YOUR CODE HERE
    raise NotImplementedError


In [ ]:
# TEST CELL: Task 1 (20 points)
points = 0
try:
    rgb = read_rgb(str(DATA_DIR/'color_blocks.png'))
    assert isinstance(rgb, np.ndarray)
    assert rgb.dtype == np.uint8
    assert rgb.ndim == 3 and rgb.shape[2] == 3
    # пиксели в разных блоках должны быть ожидаемыми в RGB
    assert (rgb[0, 10]  == np.array([255, 0, 0], dtype=np.uint8)).all()  # red
    assert (rgb[0, 80]  == np.array([0, 255, 0], dtype=np.uint8)).all()  # green
    assert (rgb[0, 150] == np.array([0, 0, 255], dtype=np.uint8)).all()  # blue
    points = 20
except Exception as e:
    print('Task 1 failed:', repr(e))
_set_score('1', points, 20)


## Task 2 (30 баллов): resize с сохранением пропорций + pad (letterbox)
Реализуйте `letterbox(img_rgb, target_hw, pad_value)`:
- сохраняет пропорции (scale = min(target_w/w, target_h/h)
- делает resize
- вставляет в центр холста размера target_hw
- паддинг заполняет `pad_value`
- возвращает `(out_img, info)` где `info` содержит `scale`, `top`, `left`, `new_hw`

In [ ]:
# TODO: implement letterbox
def letterbox(img_rgb: np.ndarray, target_hw=(224,224), pad_value=0):
    # YOUR CODE HERE
    raise NotImplementedError


In [ ]:
# TEST CELL: Task 2 (30 points)
points = 0
try:
    src = read_rgb(str(DATA_DIR/'aspect.png'))
    out, info = letterbox(src, target_hw=(224,224), pad_value=128)
    assert out.shape == (224,224,3)
    assert out.dtype == np.uint8
    assert isinstance(info, dict)
    for k in ['scale','top','left','new_hw']:
        assert k in info
    nh, nw = info['new_hw']
    assert 0 < nh <= 224 and 0 < nw <= 224
    # паддинг в углу должен быть равен pad_value
    assert (out[0,0] == np.array([128,128,128], dtype=np.uint8)).all()
    # центральная область (вставка) не должна быть полностью паддингом
    top, left = int(info['top']), int(info['left'])
    crop = out[top:top+nh, left:left+nw]
    assert crop.size > 0
    assert not np.all(crop == 128)
    points = 30
except Exception as e:
    print('Task 2 failed:', repr(e))
_set_score('2', points, 30)


## Task 3 (20 баллов): float32 CHW + нормализация
Реализуйте `to_tensor(img_rgb, mean, std)`:
- uint8 HWC (0..255) → float32 HWC (0..1)
- нормализация `(x-mean)/std` по каналам
- transpose в CHW
- вернуть `np.ndarray float32` shape `(3,H,W)`

In [ ]:
# TODO: implement to_tensor
def to_tensor(img_rgb: np.ndarray, mean=IMAGENET_MEAN, std=IMAGENET_STD) -> np.ndarray:
    # YOUR CODE HERE
    raise NotImplementedError


In [ ]:
# TEST CELL: Task 3 (20 points)
points = 0
try:
    src = read_rgb(str(DATA_DIR/'color_blocks.png'))
    out, _ = letterbox(src, target_hw=(224,224), pad_value=0)
    x = to_tensor(out)
    assert isinstance(x, np.ndarray)
    assert x.dtype == np.float32
    assert x.shape == (3,224,224)
    assert np.isfinite(x).all()
    # диапазон должен быть разумным (после нормализации обычно около [-3, +3])
    assert float(x.min()) > -10 and float(x.max()) < 10
    points = 20
except Exception as e:
    print('Task 3 failed:', repr(e))
_set_score('3', points, 20)


## Task 4 (15 баллов): собрать eval-пайплайн
Реализуйте `preprocess_eval(path, target_hw)`:
- читает RGB
- делает letterbox
- переводит в tensor через `to_tensor`
- возвращает `(x, info)` где `x` — CHW float32, `info` — словарь из letterbox

In [ ]:
# TODO: implement preprocess_eval
def preprocess_eval(path: str, target_hw=(224,224)):
    # YOUR CODE HERE
    raise NotImplementedError


In [ ]:
# TEST CELL: Task 4 (15 points)
points = 0
try:
    x, info = preprocess_eval(str(DATA_DIR/'aspect.png'), target_hw=(224,224))
    assert x.shape == (3,224,224)
    assert x.dtype == np.float32
    assert isinstance(info, dict)
    # детерминированность: два вызова подряд должны дать одинаковый результат
    x2, info2 = preprocess_eval(str(DATA_DIR/'aspect.png'), target_hw=(224,224))
    assert np.allclose(x, x2, atol=1e-6)
    assert info == info2
    points = 15
except Exception as e:
    print('Task 4 failed:', repr(e))
_set_score('4', points, 15)


## Task 5 (15 баллов): sanity-report
Реализуйте `sanity_report(arr)` и верните словарь:
`shape`, `dtype`, `min`, `max`, `mean`, `std`, `has_nan`
Это нужно, чтобы быстро ловить ошибки dtype/range/NaN до обучения.

In [ ]:
# TODO: implement sanity_report
def sanity_report(arr) -> dict:
    # YOUR CODE HERE
    raise NotImplementedError


In [ ]:
# TEST CELL: Task 5 (15 points)
points = 0
try:
    x, _ = preprocess_eval(str(DATA_DIR/'aspect.png'), target_hw=(224,224))
    rep = sanity_report(x)
    assert isinstance(rep, dict)
    for k in ['shape','dtype','min','max','mean','std','has_nan']:
        assert k in rep
    assert rep['shape'] == (3,224,224)
    assert rep['dtype'] in ('float32', 'torch.float32', 'numpy.float32') or 'float32' in str(rep['dtype'])
    assert isinstance(rep['has_nan'], bool)
    points = 15
except Exception as e:
    print('Task 5 failed:', repr(e))
_set_score('5', points, 15)


In [ ]:
# Recompute total just in case
total = float(sum(SCORES.values()))
print(f'FINAL TOTAL (without penalty): {total} / 100')


In [ ]:
def penalty_fraction(start_dt, due_dt, now_dt) -> float:
    if not (start_dt and due_dt and now_dt):
        return 0.0
    window = _sec(due_dt - start_dt)
    if window <= 0:
        return 1.0 if now_dt > due_dt else 0.0
    late = max(0.0, _sec(now_dt - due_dt))
    return min(1.0, late / window)
	
# применяем штраф
try:
    pf = penalty_fraction(start_dt, due_dt, submission_dt)
except NameError:
    from datetime import timezone
    pf = 0.0
final_score = max(0.0, total * (1.0 - min(1.0, pf)))
print(final_score)
import json
final = {
    "name": STUDENT_NAME,
    "group": GROUP,
    "assignment":ASSIGMENT,
    "score": float(total),
    "penalty_score": float(final_score),
    "due_date": DUE_DATE,
    "start_date": START_DATE,
}
print(json.dumps(final, ensure_ascii=False))